In [ ]:
import pandas as pd
import re

# Load the CSV file
data = pd.read_csv('../../../dataset/train.csv')

# Filter to only include item_weight entity
item_weight_data = data[data['entity_name'] == 'height']

# Extract numeric value and unit from entity_value
def split_value(value):
    match = re.match(r'(\d+(\.\d+)?)(\s+)?(yard|inch|meter|millimeter|foot|centimeter)', value)
    if match:
        number = float(match.group(1))
        unit = match.group(4)
        return number, unit
    return None, None

item_weight_data[['numeric_value', 'unit']] = item_weight_data['entity_value'].apply(split_value).apply(pd.Series)

# Drop rows with missing values
item_weight_data.dropna(inplace=True)

# Save processed data to a new CSV file
item_weight_data.to_csv('height.csv', index=True)

print("Data preprocessing complete. Processed data saved to 'processed_item_weight_data.csv'.")

In [2]:
from utils_height import download_images

In [ ]:
download_images(item_weight_data['image_link'].tolist(),  item_weight_data.index.tolist(), "dataset")

In [1]:
import os
import logging
import pandas as pd
from paddleocr import PaddleOCR

logging.getLogger('ppocr').setLevel(logging.ERROR)

image_folder = 'dataset/'  # Adjust this to the correct folder containing your images
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Initialize PaddleOCR
count = 0
def extract_ocr_text(index):
    image_path = os.path.join("dataset", f"{index}.jpg")
    if not os.path.exists(image_path):
        return ""
    
    # count += 1
    # if count % 100 == 0:
    #     print(f"Processed {count} images")
    result = ocr.ocr(image_path)
    ocr_text = ""
    
    # Extract text from OCR results
    for line in result:
        for box, (text, confidence) in line:
            ocr_text += text + " "
    
    return ocr_text.strip()

/home/blank/miniconda3/envs/tf/lib/python3.10/site-packages/paddle/base/framework.py:688: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(


In [2]:
extract_ocr_text(7195)

UnboundLocalError: local variable 'count' referenced before assignment

In [ ]:
for index in indices:
    print(index)
    break